The first cell is used to install and import the packages necessary in this Notebook.
* Use !pip install package, to install packages before imporing them

In [1]:
import cobra
from cobra import Model, Metabolite, Reaction
import pandas as pd
import json
import reframed as re
from cobra.flux_analysis import flux_variability_analysis
import gurobipy
from cobra.io import save_json_model

The following cell contains all the functions that are used within this Jupyter Notebook.

In [2]:
def df_to_reaction(df, model):
    """

    :param df: DataFrame with two columns.
    The first column contains metabolite IDs,
    the second column contains coefficients.
    :param model: The model that contains the metabolite objects
    :return: Returns a Reaction that contains Metabolite objects
    (gathered from model by using the metabolite IDs from df),
    with the corresponding coefficients from df.
    """

    metabolite_to_coefficient = {}  # dictionary which will be used to map from Metabolite object to coefficient (float)
    for list in df.values.tolist():
        metabolite_to_coefficient[model.metabolites.get_by_id(list[1])] = float(list[0])

    reaction = Reaction()

    reaction.add_metabolites(metabolite_to_coefficient)

    return reaction


def add_attributes(reaction, reaction_id, model):
    """

    :param reaction: Reaction object.
    :param reaction_id: Reaction ID of the reaction that we want the attributes from.
    :param model: Model which contains Reaction object with Reaction ID.
    :return: Reaction object with the same id, name, subsystem, lower- and upper bound as
    Reaction object from model with id = reaction_id
    """
    reaction.id = model.reactions.get_by_id(reaction_id).id + "_new"
    reaction.name = model.reactions.get_by_id(reaction_id).name
    reaction.subsystem = model.reactions.get_by_id(reaction_id).subsystem
    reaction.lower_bound = model.reactions.get_by_id(reaction_id).lower_bound
    reaction.upper_bound = model.reactions.get_by_id(reaction_id).upper_bound

    return reaction


def reversibility_check(reaction):
    """

    :param reaction: Reaction object.
    :return: lower and upper flux bounds of Reaction object.
    """
    return reaction.lower_bound, reaction.upper_bound


def new_synthesis_reaction(df, rxn_id, model):
    """

    :param df: DataFrame with two columns: Coefficient (mmol/gDW) and metabolite ID.
    :param rxn_id: Reaction ID of the original synthesis Reaction object.
    :param model: The metabolic model in use.
    :return: New synthesis Reaction object with the metabolites and coefficients from df.
    """
    synthesis_reaction = df_to_reaction(df, model)
    synthesis_reaction = add_attributes(synthesis_reaction, rxn_id, model)

    return synthesis_reaction

def set_exchange_bounds(list, model):
    """
    set_exchange_bounds takes in a list of compounds that should be available for the model. 
    The flux bounds are set to (-1000.0, 1000.0) (unrestricted) for the exchange reactions of 
    compounds of the list. For the exchange reactions not represented in the list, only secretion is allowed:
    flux range (0.0, 1000.0).
    """
    for exchange in model.exchanges:
        if exchange.id in list:  # if the exchange reaction is
            # represented in the medium, unrestricted uptake and secretion is allowed
            model.reactions.get_by_id(exchange.id).lower_bound = -1000.0
            model.reactions.get_by_id(exchange.id).upper_bound = 1000.0
        else:
            # if the exchange reaction is not part of the medium, the model should not be allowed to take it up,
            # but it is allowed to secrete it.
            model.reactions.get_by_id(exchange.id).lower_bound = 0.0
            model.reactions.get_by_id(exchange.id).upper_bound = 1000.0
            
def set_exchange_bounds_measured(bounds_df, model):
    """
    set_exchange_bounds_measured takes in a df with three columns: "Exchange ID", 
    "Lower bound" and "Upper bound". The function sets the flux bounds of the reactions
    represented in the "Exchange ID" column to (Lower bound, Upper bound).
    """

    exchange_ids = [id for id in bounds_df["Exchange ID"]]  # there are actually not only exchange ids,
    # there is also the methanol "exchange" reaction and the bof

    lower_bounds = [lb for lb in bounds_df["Lower bound"]]

    upper_bounds = [ub for ub in bounds_df["Upper bound"]]
    
    # setting the flux bounds: 
    i = 0
    while i < len(exchange_ids) and exchange_ids[i] != "bio00006_new":  # we dont want to change the growth rate yet
        model.reactions.get_by_id(exchange_ids[i]).upper_bound = 1000.0  # adding this line to avoid 
        #"lower bound must be lower than upper bound" error message
        model.reactions.get_by_id(exchange_ids[i]).lower_bound = lower_bounds[i]
        model.reactions.get_by_id(exchange_ids[i]).upper_bound = upper_bounds[i]
        
        i += 1

def set_bounds(model, solution):
    for exchange_ID in solution.keys():

        if solution[exchange_ID] < 0:

            model.reactions.get_by_id(exchange_ID).lower_bound = solution[exchange_ID]
            model.reactions.get_by_id(exchange_ID).upper_bound = 0.0

        else:

            model.reactions.get_by_id(exchange_ID).upper_bound = solution[exchange_ID]
            model.reactions.get_by_id(exchange_ID).lower_bound = 0.0


def write_flux_json(filename, model):
    """ write_flux_json writes a json file from a dictionary of model reactions
    and associated flux values
    """
    flux_dict = model.optimize().fluxes.to_dict()  # reaction:flux dict
    with open(filename, "w") as f:
        json.dump(flux_dict, f)


def write_min_flux_json(filename, fva_solution):
    """ write_min_flux_json writes json files from the minimal flux-values found with fva
    """
    min_flux_dict = fva_solution["minimum"].to_dict()
    with open(filename, "w") as f:
        json.dump(min_flux_dict, f)


def write_max_flux_json(filename, fva_solution):
    """ write_max_flux_json writes json files from the maximal flux-values found with fva
    """
    max_flux_dict = fva_solution["maximum"].to_dict()
    with open(filename, "w") as f:
        json.dump(max_flux_dict, f)
        


First we upload the iBsu1147 model. This model has an optimal predicted growth rate of 0.100, because the flux bounds of the BOF are "locked" to ub = lb = 0.1. When setting unrestricted flux through the biomass objective function, the model predicts a growth rate of 0.206 (h^-1). However, we must keep in mind that this is in a different nutrient environment than what we have used. That will be changed later

In [3]:
iBsu1147 = cobra.io.read_sbml_model("C:/Users/LinnS/OneDrive/5. klasse V/Python/iBSU1147_project/iBsu1147_met.xml")
print(iBsu1147.optimize())  
print(iBsu1147.reactions.get_by_id("bio00006").lower_bound, iBsu1147.reactions.get_by_id("bio00006").upper_bound)

iBsu1147.reactions.get_by_id("bio00006").lower_bound = -1000.0
iBsu1147.reactions.get_by_id("bio00006").upper_bound = 1000.0

print(iBsu1147.optimize())  

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-24
<Solution 0.100 at 0x29b234ef070>
0.1 0.1
<Solution 0.206 at 0x29b234ef070>


It turns out that the metabolite objects of the model have name and id mixed up. The following cell contains code to correct this mixup.

In [4]:
for metabolite in iBsu1147.metabolites:
    name = metabolite.id
    id = metabolite.name

    metabolite.name = name
    metabolite.id = id

In [5]:
# It was found that B. subtilis possesses two pathways for formaldehyde dissimilation: dissimilatory RuMP and
# bacillithiol (BSH) dependent formaldehyde dissimilation, however, only the former is represented in the model
# We therefore want to add the BSH-dependent pathway to the model to investigate the flux through this pathway.
# There are three reactions that need to be added to the model for this pathway. Some of the metabolites
# are already present in the model, while three new metabolites must be added. We therefore start off with creating
# these three metabolite objects, and ModelSeed information will be used, since KEGG information is not available for
# these three metabolites.

bsh = Metabolite(id="cpd26619", formula="C13H21N2O10S", name="bacillithiol | BSH", charge=-1, compartment="c")

s_hydroxymethyl_bsh = Metabolite(id="cpd32688", formula="C14H23N2O11S", name="S-(hydroxymethyl)-bacillithiol",
                                 charge=-1, compartment="c")

s_formylbacillithiol = Metabolite(id="cpd33221", formula="C14H21N2O11S", name="S-formylbacillithiol", charge=-1,
                                  compartment="c")

# Now we create Reaction objects for the three reactions we are going to add, and we add metabolites


rxn42075 = Reaction(id="rxn42075", name="Formaldehyde + BSH <--> S-(hydroxymethyl)bacillithiol",
                    lower_bound=-1000.0, upper_bound=1000.0)  # this is the default for a reversible reaction

rxn42075.add_metabolites({
    iBsu1147.metabolites.get_by_id("C00067[c]"): -1.0,
    bsh: -1.0,
    s_hydroxymethyl_bsh: 1.0
})

rxn44708 = Reaction(id="rxn44708", name="NAD + S-(hydroxymethyl)bacillithiol <--> NADH + H+ + S-formylbacillithiol",
                    lower_bound=-1000.0, upper_bound=1000.0)

rxn44708.add_metabolites({
    s_hydroxymethyl_bsh: -1.0,
    iBsu1147.metabolites.get_by_id("C00003[c]"): -1.0,
    s_formylbacillithiol: 1.0,
    iBsu1147.metabolites.get_by_id("C00004[c]"): 1.0,
    iBsu1147.metabolites.get_by_id("C00080[c]"): 1.0,

})

rxn46864 = Reaction(id="rxn46864", name="H2O + S-formylbacillithiol <--> Formate + H+ + BSH",
                    lower_bound=-1000.0, upper_bound=1000.0)

rxn46864.add_metabolites({
    s_formylbacillithiol: -1.0,
    iBsu1147.metabolites.get_by_id("C00001[c]"): -1.0,
    iBsu1147.metabolites.get_by_id("C00058[c]"): 1.0,
    iBsu1147.metabolites.get_by_id("C00080[c]"): 1.0,
    bsh: 1.0
})
# I realize there must be a synthesis reaction for BSH, otherwize flux through the first reaction will be
# impossible. By using ModelSeed I was able to backtrack the reactions that are necessary to add to the model 
# to synthesise BSH, these are rxn27003, rxn27004, rxn27005.
# First we add the two missing metabolites that are necessary in these reactions:

glcnac_mal = Metabolite(id="cpd26617", formula="C12H17NO10", charge=-2, name="GlcNAc-Mal", compartment="c")

glcn_mal = Metabolite(id="cpd26618", formula="C10H16NO9", charge=-1, name="GlcN-Mal", compartment="c")

# Then we create the reactions:

rxn27003 = Reaction(id="rxn27003", name="UDP-N-acetylglucosamine + L-Malate <--> UDP + GlcNAc-Mal",
                    lower_bound=-1000.0, upper_bound=1000.0)

rxn27003.add_metabolites({
    iBsu1147.metabolites.get_by_id("C00043[c]"): -1.0,
    iBsu1147.metabolites.get_by_id("C00149[c]"): -1.0,
    iBsu1147.metabolites.get_by_id("C00015[c]"): 1.0,
    glcnac_mal: 1.0
})

rxn27004 = Reaction(id="rxn27004", name="H2O + GlcNAc-Mal <--> Acetate + GlcN-Mal",
                    lower_bound=-1000.0, upper_bound=1000.0)

rxn27004.add_metabolites({
    glcnac_mal: -1.0,
    iBsu1147.metabolites.get_by_id("C00001[c]"): -1.0,
    iBsu1147.metabolites.get_by_id("C00164[c]"): 1.0,
    glcn_mal: 1.0
})

rxn27005 = Reaction(id="rxn27005", name="L-Cysteine + GlcN-Mal <--> H2O + BSH",
                    lower_bound=-1000.0, upper_bound=1000.0)

rxn27005.add_metabolites({
    iBsu1147.metabolites.get_by_id("C00097[c]"): -1.0,
    glcn_mal: -1.0,
    iBsu1147.metabolites.get_by_id("C00001[c]"): 1.0,
    bsh: 1.0
})

# create a list containing all the necessary metabolites
bsh_metabolites = [bsh, s_hydroxymethyl_bsh, s_formylbacillithiol, glcnac_mal, glcn_mal]


# create a list containing all the necessary reactions
# bsh_dependent_oxidation = [rxn42075, rxn44708, rxn46864, rxn27003, rxn27004, rxn27005]
# the last three are not necessary to add, because bsh is regenerated in the reaction, we therefore do not add these 
bsh_dependent_oxidation = [rxn42075, rxn44708, rxn46864]

In [6]:
# is flux possible if we add the new pathway to iBsu1147: 
iBsu1147_test = iBsu1147.copy()

iBsu1147_test.add_metabolites(bsh_metabolites)
iBsu1147_test.add_reactions(bsh_dependent_oxidation)

for reaction in bsh_dependent_oxidation:
    print(reaction.id)
    iBsu1147_test.objective = reaction.id
    print(iBsu1147_test.optimize())
    
# we also check the formate <--> co2 reaction: 

iBsu1147_test.objective = "R00519"

print(iBsu1147_test.optimize())  # we get flux through the BSH dependent oxidation pathway, 
# but not the BSH "synthesis" pathway

Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmp5xvx3oia.lp
Reading time = 0.02 seconds
: 1450 rows, 3486 columns, 14440 nonzeros
rxn42075
<Solution 63.850 at 0x29b2542b310>
rxn44708
<Solution 63.850 at 0x29b2542b100>
rxn46864
<Solution 63.850 at 0x29b2542b160>
<Solution 20.962 at 0x29b2542b370>


Creating new synthesis reactions based on experimental data:

In [7]:
# We are going to add a new synthesis reaction; cofactor synthesis.
# That means we are also introducing a new metabolite object to the model; a cofactor metabolite
# We start by creating this metabolite, and adding it to the model.
# The metabolite object has five attributes: id, name, formula, charge and compartment.

cofactor = Metabolite(id="Cofactor[c]", formula=None, name="Cofactor", charge=None,
                      compartment="c")

iBsu1147.add_metabolites([cofactor])

# Creating new synthesis reactions:

# Creating the cofactor synthesis reaction.

cofactors_and_ions = pd.read_excel("synthesis_reactions.xlsx", sheet_name="cofactors")  # DataFrame with two columns:
# one with metabolite IDs and one with coefficients.

cofactor_synthesis = df_to_reaction(cofactors_and_ions, iBsu1147)

cofactor_synthesis.name = "cofactor_synthesis"

cofactor_synthesis.id = "cofactor_synthesis"

cofactor_synthesis.lower_bound = -1000.0

# importing new DNA synthesis reaction:

DNA_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="DNA")

DNA_synthesis = new_synthesis_reaction(DNA_df, "rxn05294", iBsu1147)

# importing new mRNA synthesis reaction:

RNA_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="RNA")

RNA_synthesis = new_synthesis_reaction(RNA_df, "rxn05295", iBsu1147)

RNA_synthesis.lower_bound = -1000.0  # setting the lower bound to -1000.0,
# because the other synthesis reactions have this lower bound.
# However, as the objective is set to producing biomass,
# the flux through the synthesis reactions at optimal solution will never be negative.

# importing new lipid synthesis reaction:

lipid_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="lipid")

lipid_synthesis = new_synthesis_reaction(lipid_df, "rxn10201", iBsu1147)

# importing new lipoteichoic acid synthesis reaction:
# rxn10200

lipo_acid_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="lipoteichoic acid")

lipo_acid_synthesis = new_synthesis_reaction(lipo_acid_df, "rxn10200", iBsu1147)

# importing new cell wall synthesis reaction:
# rxn10198

cell_wall_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="cell wall")

cell_wall_synthesis = new_synthesis_reaction(cell_wall_df, "rxn10198", iBsu1147)

# importing new biomass synthesis reaction:
# bio00006

#importing new protein synthesis reaction for WTG

protein_wtg_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="protein WTG")

protein_synthesis_wtg = new_synthesis_reaction(protein_wtg_df, "rxn05296", iBsu1147)

# The only synthesis reactions that differ between the biomass samples are protein synthesis and BOF
# cell wall, lipid, lipotheichoic acid, RNA, DNA and cofactor synthesis reactions are the same

protein_wtm_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="protein WTM")

protein_synthesis_wtm = new_synthesis_reaction(protein_wtm_df, "rxn05296", iBsu1147)

# importing the protein synthesis reactions for mdhg and mdhm
# protein synthesis for mdh g:
protein_mdhg_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="protein MDHG")

protein_synthesis_mdhg = new_synthesis_reaction(protein_mdhg_df, "rxn05296", iBsu1147)

# protein synthesis for mdh m:
protein_mdhm_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="protein MDHM")

protein_synthesis_mdhm = new_synthesis_reaction(protein_mdhm_df, "rxn05296", iBsu1147)

# creating the new biomass synthesis reactions
bof_wtg_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="mBOF WTG")

bof_wtg = new_synthesis_reaction(bof_wtg_df, "bio00006", iBsu1147)

bof_wtg.lower_bound = -1000.0
bof_wtg.upper_bound = 1000.0

# importing the BOF for wt_m:
bof_wtm_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="mBOF WTM")

bof_wtm = new_synthesis_reaction(bof_wtm_df, "bio00006", iBsu1147)

bof_wtm.lower_bound = -1000.0  # setting unrestricted flux bounds for now
bof_wtm.upper_bound = 1000.0


# importing the BOF for mdh_g:
mdhg_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="mBOF MDHG")

bof_mdhg = new_synthesis_reaction(mdhg_df, "bio00006", iBsu1147)

bof_mdhg.lower_bound = -1000.0
bof_mdhg.upper_bound = 1000.0

# importing the BOF for mdh_m:
mdhm_df = pd.read_excel("synthesis_reactions.xlsx", sheet_name="mBOF MDHM")

bof_mdhm = new_synthesis_reaction(mdhm_df, "bio00006", iBsu1147)

bof_mdhm.lower_bound = -1000.0  # unrestricted flux through the BOF for now
bof_mdhm.upper_bound = 1000.0


# Creating dictionaries containing the synthesis reactions for said biomass sample
# the dictionaries will be used to add the reactions to the model



Now that all the new synthesis reactions have been made, we create model objects for each biomass sample. That way we can compare the results to the results of the original iBsu1147 model.

In [8]:
# Creating a model object for each biomass sample: 

wtg = iBsu1147.copy() 
wtm = iBsu1147.copy()
mdhg = iBsu1147.copy()
mdhm = iBsu1147.copy()

# creating a list of the old synthesis reactions. 
# Will be used to replace the reactions with new ones
synthesis_ids = ["rxn05294", "rxn05295", "rxn05296", "rxn11921", "rxn10198",
                 "rxn10200", "rxn10201", "bio00006"]  # this is a list over the old synthesis reactions that are to be replaced. 

# Creating reaction lists for each biomass sample.
# Notice that between the biomass samples, only the BOF and protein synthesis reactions differ
new_reactions_wtg = [DNA_synthesis, cofactor_synthesis,  RNA_synthesis, lipid_synthesis,
                     lipo_acid_synthesis, protein_synthesis_wtg, cell_wall_synthesis,
                     bof_wtg]

new_reactions_wtm = [DNA_synthesis, cofactor_synthesis,  RNA_synthesis, lipid_synthesis,
                     lipo_acid_synthesis, protein_synthesis_wtm, cell_wall_synthesis,
                     bof_wtm]

new_reactions_mdhg = [DNA_synthesis, cofactor_synthesis,  RNA_synthesis, lipid_synthesis,
                     lipo_acid_synthesis, protein_synthesis_mdhg, cell_wall_synthesis,
                     bof_mdhg]

new_reactions_mdhm = [DNA_synthesis, cofactor_synthesis,  RNA_synthesis, lipid_synthesis,
                     lipo_acid_synthesis, protein_synthesis_mdhm, cell_wall_synthesis,
                     bof_mdhm]

# Now we remove the old synthesis reactions from the four models.

print(len(wtg.reactions))  # just a check to see if the number of reactions changes

# removing the old reactions for all four biomass samples:
for id in synthesis_ids:
    wtg.remove_reactions([id])
    wtm.remove_reactions([id])
    mdhg.remove_reactions([id])
    mdhm.remove_reactions([id])

print(len(wtg.reactions))  # now 1735, so 8 have been removed. That corresponds to the length of synthesis_ids

# adding new reactions. This time I get no warnings of "reaction is already present"
for reaction in new_reactions_wtg:
    wtg.add_reactions([reaction])

for reaction in new_reactions_wtm:
    wtm.add_reactions([reaction])

for reaction in new_reactions_mdhg:
    mdhg.add_reactions([reaction])
    
for reaction in new_reactions_mdhm:
    mdhm.add_reactions([reaction])

print(len(wtg.reactions))  # now we have 1743 again, so eight have been added. These are:
# protein, DNA, RNA, lipid, lipoteichoic acid, cell wall, cofactors and the biomass synthesis function.

# Checking that the new reactions are there:
# for reaction in new_reactions_wtg:
#    print(iBsu1147.reactions.get_by_id(reaction.id))  # all the reactions are there with their new ids.

# setting the new objective:
wtg.objective = "bio00006_new"  # the BOFs have been given the same id, however, the reactions differ
wtm.objective = "bio00006_new"
mdhg.objective = "bio00006_new"
mdhm.objective = "bio00006_new"

print(wtg.objective)  # now the new biomass synthesis reaction is the new objective


Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmp4v92hne2.lp
Reading time = 0.00 seconds
: 1451 rows, 3486 columns, 14440 nonzeros
Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmpqtmit7ky.lp
Reading time = 0.01 seconds
: 1451 rows, 3486 columns, 14440 nonzeros
Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmpt06wvwsm.lp
Reading time = 0.01 seconds
: 1451 rows, 3486 columns, 14440 nonzeros
Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmphsc9t376.lp
Reading time = 0.01 seconds
: 1451 rows, 3486 columns, 14440 nonzeros
1743
1735
1743
Maximize
1.0*bio00006_new - 1.0*bio00006_new_reverse_d8b7f


C:\Users\LinnS\anaconda3\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [9]:
# is flux possible if we add the new pathway to iBsu1147: 
iBsu1147_test = wtg.copy()

iBsu1147_test.add_metabolites(bsh_metabolites)
iBsu1147_test.add_reactions(bsh_dependent_oxidation)

for reaction in bsh_dependent_oxidation:
    print(reaction.id)
    iBsu1147_test.objective = reaction.id
    print(iBsu1147_test.optimize())
    
# we also check the formate <--> co2 reaction: 

iBsu1147_test.objective = "R00519"

print(iBsu1147_test.optimize())  # we get flux through the BSH dependent oxidation pathway, 
# but not the BSH "synthesis" pathway

Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmpvk5oqw9y.lp
Reading time = 0.00 seconds
: 1451 rows, 3486 columns, 14390 nonzeros
rxn42075
<Solution 63.850 at 0x29b2b93dd80>
rxn44708
<Solution 63.850 at 0x29b2b93dc60>
rxn46864
<Solution 63.850 at 0x29b2b93dde0>
<Solution 20.962 at 0x29b2b93dba0>


Now we have one model object for each biomass sample, in addition to the original model. We want to conduct a test to see how the new synthesis reactions affect the predicted growth rate.

Now that the models have been updated with new synthesis reactions, it is time to change the flux bounds of each model to represent the values that were measured experimentally. 

In [10]:
# importing df that contains the components of the minimal medium used experimentally
experimental_medium_df = pd.read_excel("Medium components.xlsx")

experimental_medium_list = []

# for some reason i did not manage to use the column of the df directly,
# so I am making a list containing all the exchange ids representing the medium

for id in experimental_medium_df["Exchange reaction ID"]:
    experimental_medium_list.append(id)

print(experimental_medium_list)

# for now we want to set the flux range to -1000 and 1000 for the exchanges that represent compounds and gases that have been
# available for the cultivated bacteria.
# later, we want to set the flux-ranges to represent the secretion and uptake rates of components that we have measured.

set_exchange_bounds(experimental_medium_list, iBsu1147)
set_exchange_bounds(experimental_medium_list, wtg)
set_exchange_bounds(experimental_medium_list, wtm)
set_exchange_bounds(experimental_medium_list, mdhg)
set_exchange_bounds(experimental_medium_list, mdhm)

# for now we set the methanol uptake to 0. This has to be done seperately, 
# because EX_Methanol is not an exchange reaction, and is therefore not captured by 
# set_exchange_bounds. (if we used "reaction" instead of "exchange"
# in the set_exchange_bounds function, the methanol uptake would have been covered, 
# however, we would then mess with the flux bounds of all reactions in the model, 
# which is not what we want)

iBsu1147.reactions.get_by_id("EX_Methanol[c]").lower_bound = 0.0
wtg.reactions.get_by_id("EX_Methanol[c]").lower_bound = 0.0
wtm.reactions.get_by_id("EX_Methanol[c]").lower_bound = 0.0
mdhg.reactions.get_by_id("EX_Methanol[c]").lower_bound = 0.0
mdhm.reactions.get_by_id("EX_Methanol[c]").lower_bound = 0.0

# restricting the L-trp uptake:
iBsu1147.reactions.get_by_id("E00032").lower_bound = -0.5
wtg.reactions.get_by_id("E00032").lower_bound = -0.5
wtm.reactions.get_by_id("E00032").lower_bound = -0.5
mdhg.reactions.get_by_id("E00032").lower_bound = -0.5
mdhm.reactions.get_by_id("E00032").lower_bound = -0.5

# restricting the L-glu uptake:
iBsu1147.reactions.get_by_id("E00009").lower_bound = -1.0
wtg.reactions.get_by_id("E00009").lower_bound = -1.0
wtm.reactions.get_by_id("E00009").lower_bound = -1.0
mdhg.reactions.get_by_id("E00009").lower_bound = -1.0
mdhm.reactions.get_by_id("E00009").lower_bound = -1.0

# We also restrict the uptake of glucose which is the carbon source in this case
# otherwize the predicted growth rates would be unreasonably high

iBsu1147.reactions.get_by_id("E00096").lower_bound = -1.8
wtg.reactions.get_by_id("E00096").lower_bound = -1.8
wtm.reactions.get_by_id("E00096").lower_bound = -1.8
mdhg.reactions.get_by_id("E00096").lower_bound = -1.8
mdhm.reactions.get_by_id("E00096").lower_bound = -1.8

# we check the predicted growth rate again
print(iBsu1147.optimize()) 
print(wtg.optimize())
print(wtm.optimize())
print(mdhg.optimize())
print(mdhm.optimize())



['E00160', '-', 'E00006', 'E00003', 'E00084', '-', 'E00096', 'E00032', 'E00009', 'E00103', 'E00023', 'E00217', 'E00015', 'E00030', 'E00027', 'E00013', 'E00067', 'E00200', 'E00002', 'E00001', 'E00033']
<Solution 0.216 at 0x29b2b93dc90>
<Solution 0.224 at 0x29b2b93ff40>
<Solution 0.219 at 0x29b2b93f7c0>
<Solution 0.224 at 0x29b2b93f040>
<Solution 0.224 at 0x29b2b93e620>


In [11]:
# is flux possible if we add the new pathway to iBsu1147: 
iBsu1147_test = wtg.copy()

iBsu1147_test.add_metabolites(bsh_metabolites)
iBsu1147_test.add_reactions(bsh_dependent_oxidation)

for reaction in bsh_dependent_oxidation:
    print(reaction.id)
    iBsu1147_test.objective = reaction.id
    print(iBsu1147_test.optimize())
    
# we also check the formate <--> co2 reaction: 

iBsu1147_test.objective = "R00519"

print(iBsu1147_test.optimize())  # we get flux through the BSH dependent oxidation pathway, 
# but not the BSH "synthesis" pathway

Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmpjbip9u4w.lp
Reading time = 0.00 seconds
: 1451 rows, 3486 columns, 14390 nonzeros
rxn42075
<Solution 70.050 at 0x29b24914af0>
rxn44708
<Solution 70.050 at 0x29b24917fa0>
rxn46864
<Solution 70.050 at 0x29b24915690>
<Solution 24.412 at 0x29b24916b60>


Apparently there are some small differences in growth rate between the models, when the medium composition is kept constant. 

Now we want to see whether the old model (iBsu1147) or the new models predict a more accurate growth rate in each case: 

In [12]:
# Uptake- and secretion rates.xlsx contains a sheet for each biomass sample. 
# We create a df for each biomass sample: 

bounds_wtg = pd.read_excel("Uptake- and secretion rates.xlsx", sheet_name="WTG")
bounds_wtm = pd.read_excel("Uptake- and secretion rates.xlsx", sheet_name="WTM")
bounds_mdhg = pd.read_excel("Uptake- and secretion rates.xlsx", sheet_name="MDHG")
bounds_mdhm = pd.read_excel("Uptake- and secretion rates.xlsx", sheet_name="MDHM")

# we set the flux bounds for to match what was measured experimentally:

set_exchange_bounds_measured(bounds_wtg, wtg)
set_exchange_bounds_measured(bounds_wtm, wtm)
set_exchange_bounds_measured(bounds_mdhg, mdhg)
set_exchange_bounds_measured(bounds_mdhm, mdhm)

# now we check the growth rate of the biomass samples: 

print("Predicted growth rates")
print(wtg.optimize())
print(wtm.optimize())
print(mdhg.optimize())
print(mdhm.optimize())

# something goes wrong when setting the bounds of wtg. 
# we therefore relax the bounds and let them go from (0.0, measured) if secretion
# and (measured, 0.0) if uptake

wtg.reactions.get_by_id("E00002").upper_bound = 0.0
wtg.reactions.get_by_id("E00004").lower_bound = 0.0
wtg.reactions.get_by_id("E00012").lower_bound = 0.0
wtg.reactions.get_by_id("E00096").upper_bound = 0.0
wtg.reactions.get_by_id("E00009").upper_bound = 0.0
wtg.reactions.get_by_id("E00032").upper_bound = 0.0
wtg.reactions.get_by_id("EX_Methanol[c]").lower_bound = 0.0
wtg.reactions.get_by_id("EX_Methanol[c]").upper_bound = 0.0

# we print to double check that the fluxes of the optimal solution match 
# the bounds in bounds_wtg, and they do.
"""
print(wtg.optimize().fluxes["E00002"],
     wtg.optimize().fluxes["E00004"],
     wtg.optimize().fluxes["E00012"],
     wtg.optimize().fluxes["E00096"],
     wtg.optimize().fluxes["E00009"],
     wtg.optimize().fluxes["E00032"],
     wtg.optimize().fluxes["EX_Methanol[c]"])
"""
print("WTG again")
print(wtg.optimize())

# we conduct a pairwize comparison of the predicted growth rates: 
# first we compare iBsu1147 and wtg

set_exchange_bounds_measured(bounds_wtg, iBsu1147)  

iBsu1147.reactions.get_by_id("E00002").upper_bound = 0.0
iBsu1147.reactions.get_by_id("E00004").lower_bound = 0.0
iBsu1147.reactions.get_by_id("E00012").lower_bound = 0.0
iBsu1147.reactions.get_by_id("E00096").upper_bound = 0.0
iBsu1147.reactions.get_by_id("E00009").upper_bound = 0.0
iBsu1147.reactions.get_by_id("E00032").upper_bound = 0.0
iBsu1147.reactions.get_by_id("EX_Methanol[c]").lower_bound = 0.0
iBsu1147.reactions.get_by_id("EX_Methanol[c]").upper_bound = 0.0
# now wtg and iBsu1147 have 
# the same flux bounds

print("iBsu1147: ", iBsu1147.optimize(), " wtg: ", wtg.optimize())

# then we compare iBsu1147 and wtm
set_exchange_bounds_measured(bounds_wtm, iBsu1147)

print("iBsu1147: ", iBsu1147.optimize(), " wtm: ", wtm.optimize())

# iBsu1147 and mdhg

set_exchange_bounds_measured(bounds_mdhg, iBsu1147)

print("iBsu1147: ", iBsu1147.optimize(), " mdhg: ", mdhg.optimize())

# iBsu1147 and mdhm

set_exchange_bounds_measured(bounds_mdhm, iBsu1147)

print("iBsu1147: ", iBsu1147.optimize(), " mdhm: ", mdhm.optimize())


Predicted growth rates
<Solution infeasible at 0x29b2b93f9a0>
<Solution 0.437 at 0x29b2b93f4f0>
<Solution 0.188 at 0x29b2b93fb50>
<Solution 0.113 at 0x29b2b93f490>
WTG again
<Solution 0.234 at 0x29b2b93f4f0>


C:\Users\LinnS\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


iBsu1147:  <Solution 0.231 at 0x29b2b93f820>  wtg:  <Solution 0.234 at 0x29b24cff880>
iBsu1147:  <Solution 0.438 at 0x29b2b93fc70>  wtm:  <Solution 0.437 at 0x29b24cff880>
iBsu1147:  <Solution 0.188 at 0x29b2b93f160>  mdhg:  <Solution 0.188 at 0x29b24cff6d0>
iBsu1147:  <Solution 0.112 at 0x29b2b93fc10>  mdhm:  <Solution 0.113 at 0x29b24cfff10>


In [13]:
# is flux possible if we add the new pathway to iBsu1147: 
iBsu1147_test = wtg.copy()

iBsu1147_test.add_metabolites(bsh_metabolites)
iBsu1147_test.add_reactions(bsh_dependent_oxidation)

for reaction in bsh_dependent_oxidation:
    print(reaction.id)
    iBsu1147_test.objective = reaction.id
    print(iBsu1147_test.optimize())
    
# we also check the formate <--> co2 reaction: 

iBsu1147_test.objective = "R00519"

print(iBsu1147_test.optimize())  # we get flux through the BSH dependent oxidation pathway, 
# but not the BSH "synthesis" pathway

Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmpokkkt_dn.lp
Reading time = 0.02 seconds
: 1451 rows, 3486 columns, 14390 nonzeros
rxn42075
<Solution 34.954 at 0x29b2cc0bdf0>
rxn44708
<Solution 34.954 at 0x29b2cc0bcd0>
rxn46864
<Solution 34.954 at 0x29b2cd000d0>
<Solution 13.225 at 0x29b2cc941f0>


According to the fba results, the predicted growth rates are almost identical for all four conditions. That suggests that even though there are significant differences in the biomass composition and protein composition, that does not affect the predicted growth rate.

Now we want to investigate the flux distribution in each condition. For WTM and MDHG the predicted growth rate is higher than the measured growth rate. In these two cases we are able to set the bounds of the BOF and run FVA directly. However, we see that for WTG and MDHM, the predicted growth rates are lower than the measured growth rates. For these cases we conduct MOMA to find the secretion and uptake rates that are as close to the measured values as possible. Then we use the secretion and uptake rates suggested by MOMA to run FVA.

In [14]:
# we start with WTM and MDHG where we can set bounds of the BOF and conduct fva directly.

wtm.reactions.get_by_id("bio00006_new").lower_bound = 0.35-(2*0.05)

wtm.reactions.get_by_id("bio00006_new").upper_bound = 0.35+(2*0.05)

#fva_wtm = flux_variability_analysis(model=wtm, processes=1, fraction_of_optimum=0.95)

# mdhg: 
mdhg.reactions.get_by_id("bio00006_new").lower_bound = 0.13 - (2 * 0.03)

mdhg.reactions.get_by_id("bio00006_new").upper_bound = 0.13 + (2 * 0.03)

#fva_mdhg = flux_variability_analysis(model=mdhg, processes=1, fraction_of_optimum=0.95)

In [15]:
# is flux possible if we add the new pathway to iBsu1147: 
iBsu1147_test = wtm.copy()


iBsu1147_test.add_metabolites(bsh_metabolites)
iBsu1147_test.add_reactions(bsh_dependent_oxidation)

for reaction in bsh_dependent_oxidation:
    print(reaction.id)
    iBsu1147_test.objective = reaction.id
    print(iBsu1147_test.optimize())
    
# we also check the formate <--> co2 reaction: 

iBsu1147_test.objective = "R00519"

print(iBsu1147_test.optimize())  # we get flux through the BSH dependent oxidation pathway, 
# but not the BSH "synthesis" pathway

Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmpsxr39u6b.lp
Reading time = 0.02 seconds
: 1451 rows, 3486 columns, 14388 nonzeros
rxn42075
<Solution 29.669 at 0x29b2b713ca0>
rxn44708
<Solution 29.669 at 0x29b2b713a30>
rxn46864
<Solution 29.669 at 0x29b2b713b80>
<Solution 14.370 at 0x29b2b713d00>


for WTG and MDHM we want to find new secretion and uptake rates that are high enough to maintain the measured growth rate. In order to do that, we use MOMA, but then it is necessary to reset the flux bounds to the default (-1000, 1000) or (0, 1000) bounds. 

In [16]:
# we need to set the flux bounds so that there is unrestricted access to medium compounds and gas
# we start with the wtg fluxes:
default_flux_df = {"E00002":[-1000.0, 1000.0], "E00004": [0.0, 1000.0], "E00009": [-1000.0, 1000.0],
           "E00012": [0.0, 1000.0], "E00032": [-1000.0, 1000.0], "E00096": [-1000.0, 1000.0]}

for exchange in wtg.exchanges:
    if exchange.id in default_flux_df.keys():
        exchange.lower_bound = default_flux_df[exchange.id][0]
        exchange.upper_bound = default_flux_df[exchange.id][1]

wtg.reactions.get_by_id("EX_Methanol[c]").lower_bound = 0.0
wtg.reactions.get_by_id("EX_Methanol[c]").upper_bound = 0.0

# now we change the mdhm fluxes: 

for exchange in mdhm.exchanges:
    if exchange.id in default_flux_df.keys():
        exchange.lower_bound = default_flux_df[exchange.id][0]
        exchange.upper_bound = default_flux_df[exchange.id][1]

mdhm.reactions.get_by_id("EX_Methanol[c]").lower_bound = -1000.0
mdhm.reactions.get_by_id("EX_Methanol[c]").upper_bound = 1000.0

# now we run MOMA. We are using the reframed package which allows us to pass in a refrence dictionary. 
# to use reframed.MOMA we need to convert the model to a cb model:
cb_wtg = re.from_cobrapy(wtg)

reference_wtg = {"E00002": -5.2, "E00004": 5.2, "E00012": 3.8, "E00096": -3.0, "E00032": -0.1, "E00009": -1.7}

reactions_wtg = ["E00002", "E00004", "E00012", "E00096", "E00032", "E00009"]

# we also set the flux bounds of the BOF: 

wtg.reactions.get_by_id("bio00006_new").lower_bound = 0.36 - 2*(0.03)

wtg.reactions.get_by_id("bio00006_new").upper_bound = 0.36 + 2*(0.03)

MOMA_wtg = re.MOMA(model=cb_wtg, reference=reference_wtg, reactions=reactions_wtg)
#print(MOMA_wtg.show_values())

# print(MOMA_wtg.show_values())

# The results are as follows:

# "E00002": -7.02987
# "E00004": 5.79471
# "E00009": -1.79149
# "E00012": 3.8
# "E00032": -0.0140311
# "E00096": -4.60114

# We set the flux bounds of wtg to the fluxes from the MOMA-result, and then we conduct fva
wtg_MOMA_results = {"E00002": -7.02987, "E00004": 5.79471, "E00009": -1.7915, "E00012": 3.8, "E00032": -0.0140311,
                     "E00096": -4.60114}  # we rounded the value for glucose uptake

# Setting the flux bounds to the fluxes suggested by MOMA
# for secretion: (0, moma_flux) for uptake (moma_flux, 0)
set_bounds(wtg, wtg_MOMA_results)

print("WTG optimal growth rates with MOMA_results")
print(wtg.optimize())


#fva_wtg = flux_variability_analysis(model=wtg, processes=1, fraction_of_optimum=0.95)

# we do the same for MDHM:
cb_mdhm = re.from_cobrapy(mdhm)

reference_mdhm = {"E00002": -3.58, "E00004": 3.60, "E00012": 2.17, "E00096": -1.29, "E00032": -0.003, "E00009": -0.65,
                 "EX_Methanol[c]": -2.15}

reactions_mdhm = ["E00002", "E00004", "E00012", "E00096", "E00032", "E00009", "EX_Methanol[c]"]

# setting the growth rate restrictions:
wtg.reactions.get_by_id("bio00006_new").lower_bound = 0.19 - 2*(0.02)

wtg.reactions.get_by_id("bio00006_new").upper_bound = 0.19 + 2*(0.02)

MOMA_mdhm = re.MOMA(model=cb_mdhm, reference=reference_mdhm, reactions=reactions_mdhm)


#print(MOMA_mdhm.show_values())

# The results are as follows: 
# "E00002": -4.57668
# "E00004": 3.79934
# "E00009": -0.824419
# "E00012": 2.29459
# "E00032": -0.00680136
# "E00096": -2.1621
# "EX_Methanol[c]": -2.28289

# We set the flux bounds of mdhm to the fluxes from the MOMA-result, and then we conduct fva
# first creating a dict that contains the moma-results we are interested in for mdhm
mdhm_MOMA_results = {"E00002": -4.57668, "E00004": 3.79934, "E00009": -0.824419, "E00012": 2.29459, "E00032": -0.00680136,
                     "E00096": -2.2, "EX_Methanol[c]": -2.28289}   # i have rounded the value for glucose uptake
# to avoid solver errors

# then we set the flux bounds to match the moma results:
set_bounds(mdhm, mdhm_MOMA_results)

"""for reaction in mdhm.reactions:
    if reaction.id in mdhm_MOMA_results.keys():
        reaction.lower_bound = mdhm_MOMA_results[reaction.id]
        reaction.upper_bound = mdhm_MOMA_results[reaction.id]
"""
# then we check if we get a feasible solution:

print("optimal solution for mdhm now:")
print(mdhm.optimize())  

#fva_mdhm = flux_variability_analysis(model=mdhm, processes=1, fraction_of_optimum=0.95)


#print("\nFVA MDHM\n")
#print(fva_mdhm)  # numeric instability for R01827 and R04173. R01827 is part of the assimilatory RuMP pathway,
# so we are interested in knowing the flux through this pathway


WTG optimal growth rates with MOMA_results
<Solution 0.300 at 0x29b2b0c4310>
optimal solution for mdhm now:
<Solution 0.151 at 0x29b2c216680>


In [17]:
# is flux possible if we add the new pathway to iBsu1147: 
iBsu1147_test = wtg.copy()

iBsu1147_test.add_metabolites(bsh_metabolites)
iBsu1147_test.add_reactions(bsh_dependent_oxidation)

for reaction in bsh_dependent_oxidation:
    print(reaction.id)
    iBsu1147_test.objective = reaction.id
    print(iBsu1147_test.optimize())
    
# we also check the formate <--> co2 reaction: 

iBsu1147_test.objective = "R00519"

print(iBsu1147_test.optimize())  # we get flux through the BSH dependent oxidation pathway, 
# but not the BSH "synthesis" pathway

Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmp3ybi072n.lp
Reading time = 0.01 seconds
: 1451 rows, 3486 columns, 14390 nonzeros
rxn42075
<Solution 26.514 at 0x29b2422c5b0>
rxn44708
<Solution 26.514 at 0x29b2422c700>
rxn46864
<Solution 26.514 at 0x29b2422f7c0>
<Solution 11.139 at 0x29b2422c640>


In [18]:
# when we try to run loopless fva, we get a runtime error that says rxn05294 not part of a model. 
print(wtg.reactions.get_by_id("rxn05294_new").lower_bound, wtg.reactions.get_by_id("rxn05294_new").upper_bound)
print(wtg.reactions.get_by_id("rxn05294_new").subsystem)

-1000.0 1000.0
Sulfur Metabolism


In [19]:
# We also want to look at the fva_results when loopless = True
# fva_wtg_loopless = flux_variability_analysis(model=wtg, processes=1, loopless=True, fraction_of_optimum=0.1)
#fva_wtm_loopless = flux_variability_analysis(model=wtm, processes=1, loopless=True, fraction_of_optimum=0.5)
#fva_mdhg_loopless = flux_variability_analysis(model=mdhg, processes=1, loopless=True, fraction_of_optimum=0.5)
#fva_mdhm_loopless = flux_variability_analysis(model=mdhm, processes=1, loopless=True, fraction_of_optimum=0.5)

In [20]:
# We do not want FVA to be loopless after all, 

fva_wtg = flux_variability_analysis(model=wtg, processes=1,  fraction_of_optimum=0.95)
fva_wtm = flux_variability_analysis(model=wtm, processes=1,  fraction_of_optimum=0.95)
fva_mdhg = flux_variability_analysis(model=mdhg, processes=1,  fraction_of_optimum=0.95)
fva_mdhm = flux_variability_analysis(model=mdhm, processes=1,  fraction_of_optimum=0.95)

In [21]:
wtg.reactions.rxn05294_new

Reaction identifier,rxn05294_new
Name,rxn05294
Memory address,0x29b23f8d870
Stoichiometry,0.925416339507073 C00131[c] + 0.700552731219607 C00286[c] + 0.699715250369375 C00458[c] + 0.921961730999863 C00459[c] <=> 3.24764605209592 C00013[c] + C00039[c] 0.925416339507073 dATP[c] + 0.700552731219607 dGTP[c] + 0.699715250369375 dCTP[c] + 0.921961730999863 dTTP[c] <=> 3.24764605209592 Pyrophosphate[c] + DNA[c]
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [22]:

# We write the results to excel and json files 
# so that we can look at them later and visualize in escher.

# Writing to excel: 
# First the regular FVA results:
with pd.ExcelWriter("FVA.xlsx") as writer:

    fva_wtg.to_excel(writer, sheet_name="WTG")

    fva_wtm.to_excel(writer, sheet_name="WTM")

    fva_mdhg.to_excel(writer, sheet_name="MDHG")

    fva_mdhm.to_excel(writer, sheet_name="MDHM")

"""
# Then the loopless FVA results: 
with pd.ExcelWriter("FVA_loopless.xlsx") as writer:

    fva_wtg_loopless.to_excel(writer, sheet_name="WTG")

    fva_wtm_loopless.to_excel(writer, sheet_name="WTM")

    fva_mdhg_loopless.to_excel(writer, sheet_name="MDHG")

    fva_mdhm_loopless.to_excel(writer, sheet_name="MDHM")

"""
# Now we write the fva results to json files to be visualized in escher
#WTG


write_min_flux_json(filename="wtg_min.json", fva_solution=fva_wtg)

write_max_flux_json(filename="wtg_max.json", fva_solution=fva_wtg)

#write_min_flux_json(filename="wtg_min_flux_ll.json", fva_solution=fva_wtg_loopless)

#write_max_flux_json(filename="wtg_max_flux_ll.json", fva_solution=fva_wtg_loopless)

# WTM 

write_min_flux_json(filename="wtm_min.json", fva_solution=fva_wtm)

write_max_flux_json(filename="wtm_max.json", fva_solution=fva_wtm)

#write_min_flux_json(filename="wtm_min_flux_ll.json", fva_solution=fva_wtm_loopless)

#write_max_flux_json(filename="wtm_max_flux_ll.json", fva_solution=fva_wtm_loopless)

# MDHG:   

write_min_flux_json(filename="mdhg_min.json", fva_solution=fva_mdhg)

write_max_flux_json(filename="mdhg_max.json", fva_solution=fva_mdhg)

#write_min_flux_json(filename="mdhg_min_flux_ll.json", fva_solution=fva_mdhg_loopless)

#write_max_flux_json(filename="mdhg_max_flux_ll.json", fva_solution=fva_mdhg_loopless)

# MDHM:

write_min_flux_json(filename="mdhm_min.json", fva_solution=fva_mdhm)

write_max_flux_json(filename="mdhm_max.json", fva_solution=fva_mdhm)

#write_min_flux_json(filename="mdhm_min_flux_ll.json", fva_solution=fva_mdhm_loopless)

#write_max_flux_json(filename="mdhm_max_flux_ll.json", fva_solution=fva_mdhm_loopless)


Now we have run flux variance analysis for all four biomass samples. However, when we visualize them in the current flux map, we see that only the assimilatory- and dissimilatory RuMP pathways are present. B. subtilis also has a third way of dealing with the formaldehyde, which is Bacillithiol dependent oxidation of formaldehyde to CO2. We are interested in seeing the flux distribution through all three pathways, and we therefore decided to add the linear oxidation of formaldehyde to the models. 

In [23]:

# want to know the predicted growth rate before adding the new reaction
print("Growth rates before adding BSH dependent oxidation")
print(wtg.optimize())
print(wtm.optimize())
print(mdhg.optimize())
print(mdhm.optimize())

# making a test: 
wtg_test = wtg.copy()
# is flux possible if we add the new pathway to iBsu1147: 
wtg_test.add_metabolites(bsh_metabolites)
wtg_test.add_reactions(bsh_dependent_oxidation)

for reaction in bsh_dependent_oxidation:
    print(reaction.id)
    wtg_test.objective = reaction.id
    print(wtg_test.optimize())
    
# we also check the formate <--> co2 reaction: 

wtg_test.objective = "R00519"

print(wtg_test.optimize())

# is flux possible if we add the new pathway to iBsu1147: 
wtg.add_metabolites(bsh_metabolites)
wtg.add_reactions(bsh_dependent_oxidation)

for reaction in bsh_dependent_oxidation:
    wtg.objective = reaction.id
    print(wtg.objective)
    print(wtg.optimize())
    
# we also check the formate <--> co2 reaction: 

wtg.objective = "R00519"

print(wtg.optimize())  # seems to work, we try for the rest as well: 


wtm.add_metabolites(bsh_metabolites)
wtm.add_reactions(bsh_dependent_oxidation)

for reaction in bsh_dependent_oxidation:
    wtm.objective = reaction.id
    print(wtm.objective)
    print(wtm.optimize())
    
# we also check the formate <--> co2 reaction: 

wtm.objective = "R00519"

print(wtm.optimize())

mdhg.add_metabolites(bsh_metabolites)
mdhg.add_reactions(bsh_dependent_oxidation)
for reaction in bsh_dependent_oxidation:
    mdhg.objective = reaction.id
    print(mdhg.objective)
    print(mdhg.optimize())
    
# we also check the formate <--> co2 reaction: 

mdhg.objective = "R00519"

print(mdhg.optimize())

mdhm.add_metabolites(bsh_metabolites)
mdhm.add_reactions(bsh_dependent_oxidation)

mdhm.add_metabolites(bsh_metabolites)
mdhm.add_reactions(bsh_dependent_oxidation)
for reaction in bsh_dependent_oxidation:
    mdhm.objective = reaction.id
    print(mdhm.objective)
    print(mdhm.optimize())
    
# we also check the formate <--> co2 reaction: 

mdhm.objective = "R00519"

print(mdhm.optimize())

# okay so apparently it works now, but I have no idea what I did differently. 
# anyway before we can run new FVA analyses, I need to change the objective back to BOF: 

wtg.objective = "bio00006_new"
wtm.objective = "bio00006_new"
mdhg.objective = "bio00006_new"
mdhm.objective = "bio00006_new"

print(wtg.optimize(),
wtm.optimize(),
mdhg.optimize(),
mdhm.optimize())  # same growth rates as before


Growth rates before adding BSH dependent oxidation
<Solution 0.230 at 0x29b2422ec20>
<Solution 0.437 at 0x29b2422ce20>
<Solution 0.188 at 0x29b2422f910>
<Solution 0.151 at 0x29b2422fdf0>
Read LP format model from file C:\Users\LinnS\AppData\Local\Temp\tmp86ei4jry.lp
Reading time = 0.01 seconds
: 1451 rows, 3486 columns, 14390 nonzeros
rxn42075
<Solution 26.514 at 0x29b24ede650>
rxn44708
<Solution 26.514 at 0x29b24ede320>
rxn46864
<Solution 26.514 at 0x29b24ede7d0>
<Solution 11.139 at 0x29b24ede8f0>
Maximize
1.0*rxn42075 - 1.0*rxn42075_reverse_3e23f
<Solution 26.514 at 0x29b24edffd0>
Maximize
1.0*rxn44708 - 1.0*rxn44708_reverse_59360
<Solution 26.514 at 0x29b24edec50>
Maximize
1.0*rxn46864 - 1.0*rxn46864_reverse_d196d
<Solution 26.514 at 0x29b24edfe80>
<Solution 11.139 at 0x29b24ede9e0>
Maximize
1.0*rxn42075 - 1.0*rxn42075_reverse_3e23f


Ignoring reaction 'rxn42075' since it already exists.
Ignoring reaction 'rxn44708' since it already exists.
Ignoring reaction 'rxn46864' since it already exists.


<Solution 29.669 at 0x29b2c2b0e50>
Maximize
1.0*rxn44708 - 1.0*rxn44708_reverse_59360
<Solution 29.669 at 0x29b2c2b0fa0>
Maximize
1.0*rxn46864 - 1.0*rxn46864_reverse_d196d
<Solution 29.669 at 0x29b2c2b0730>
<Solution 14.370 at 0x29b2c2b12d0>
Maximize
1.0*rxn42075 - 1.0*rxn42075_reverse_3e23f
<Solution 9.942 at 0x29b2c2b2350>
Maximize
1.0*rxn44708 - 1.0*rxn44708_reverse_59360
<Solution 9.942 at 0x29b2c2b3070>
Maximize
1.0*rxn46864 - 1.0*rxn46864_reverse_d196d
<Solution 9.942 at 0x29b2c2b1c00>
<Solution 5.428 at 0x29b2c2b20b0>
Maximize
1.0*rxn42075 - 1.0*rxn42075_reverse_3e23f
<Solution 22.739 at 0x29b2c2b3550>
Maximize
1.0*rxn44708 - 1.0*rxn44708_reverse_59360
<Solution 22.739 at 0x29b2c2b0ac0>
Maximize
1.0*rxn46864 - 1.0*rxn46864_reverse_d196d
<Solution 22.739 at 0x29b2c2b0d90>
<Solution 8.814 at 0x29b2c2b3fa0>
<Solution 0.230 at 0x29b2c2b07c0> <Solution 0.440 at 0x29b2c2b1ba0> <Solution 0.190 at 0x29b2c2b25f0> <Solution 0.151 at 0x29b2ca34eb0>


Now we have added the missing formaldehyde oxidation pathway, and we are interested in seeing if FVA suggests flux in this pathway or not. We therefore run new FVA analyses.

In [24]:
# we conduct FVA for each sample: 

fva_bsh_wtg = flux_variability_analysis(model=wtg, processes=1,  
                                        fraction_of_optimum=0.95)

fva_bsh_wtm = flux_variability_analysis(model=wtm, processes=1,  
                                        fraction_of_optimum=0.95)

fva_bsh_mdhg = flux_variability_analysis(model=mdhg, processes=1,  
                                        fraction_of_optimum=0.95)

fva_bsh_mdhm = flux_variability_analysis(model=mdhm, processes=1,  
                                        fraction_of_optimum=0.95)


with pd.ExcelWriter("FVA_bsh.xlsx") as writer:

    fva_bsh_wtg.to_excel(writer, sheet_name="WTG")

    fva_bsh_wtm.to_excel(writer, sheet_name="WTM")

    fva_bsh_mdhg.to_excel(writer, sheet_name="MDHG")

    fva_bsh_mdhm.to_excel(writer, sheet_name="MDHM")


# Now we write the fva results to json files to be visualized in escher
#WTG 

# writing the wtg fva solutions to json
write_min_flux_json(filename="wtg_bsh_min.json", fva_solution=fva_bsh_wtg)

write_max_flux_json(filename="wtg_bsh_max.json", fva_solution=fva_bsh_wtg)

# writing the wtm fva solutions to json
write_min_flux_json(filename="wtm_bsh_min.json", fva_solution=fva_bsh_wtm)

write_max_flux_json(filename="wtm_bsh_max.json", fva_solution=fva_bsh_wtm)

# writing the mdhg fva solutions to json
write_min_flux_json(filename="mdhg_bsh_min.json", fva_solution=fva_bsh_mdhg)

write_max_flux_json(filename="mdhg_bsh_max.json", fva_solution=fva_bsh_mdhg)

# writing the mdhm fva solutions to json
write_min_flux_json(filename="mdhm_bsh_min.json", fva_solution=fva_bsh_mdhm)

write_max_flux_json(filename="mdhm_bsh_max.json", fva_solution=fva_bsh_mdhm)


In [25]:
iBsu1147.add_metabolites(bsh_metabolites)
iBsu1147.add_reactions(bsh_dependent_oxidation)

save_json_model(iBsu1147, "iBsu1147_bsh.json")

In [26]:
print(iBsu1147.reactions.get_by_id("rxn44708"))

rxn44708: C00003[c] + cpd32688 <=> C00004[c] + C00080[c] + cpd33221


I want to know if the models suggest that there must be flux through any of the reactions of the formaldehyde metabolism pathways. 

In [27]:
bsh_reactions = ["rxn42075", "rxn44708", "rxn46864", "R00519"]

dis_RuMP = ["R03321", "R02736", "R02035", "R01528"]

ass_RuMP = ["R05338", "R05339", "R04779", "R01070", "R01830", "R01827", "R01641", "R01529", "R01056"]



In [28]:
print(fva_bsh_wtg)

# We can start off with converting each fva solution to a dataframe, with key = reaction id and value = flux range


bsh_wtg = {}

def flux_range_selection(fva_df, reaction_list):
    """
    fva_of_reactions_in_list returns the fluxranges of the reactions in reaction_list in a dictionary, 
    where key = reaction id and value = [range]
    """
    reaction_dict = {}
    for index, row in fva_df.iterrows():
        if index in reaction_list:
            if (row["minimum"] < 0.0 and row["maximum"] < 0.0) or (row["minimum"] > 0.0 and row["maximum"] > 0.0):
                reaction_dict[index] = [row["minimum"], row["maximum"]]
            
    return reaction_dict 
            
        


# For all the models, the flux range of all the reactions in the bsh_dependent pathway span 0 
print(flux_range_selection(fva_bsh_wtg, bsh_dependent_oxidation))
print(flux_range_selection(fva_bsh_wtm, bsh_dependent_oxidation))
print(flux_range_selection(fva_bsh_mdhg, bsh_dependent_oxidation))
print(flux_range_selection(fva_bsh_mdhm, bsh_dependent_oxidation))

print(flux_range_selection(fva_bsh_wtg, dis_RuMP))
print(flux_range_selection(fva_bsh_wtm, dis_RuMP))
print(flux_range_selection(fva_bsh_mdhg, dis_RuMP))
print(flux_range_selection(fva_bsh_mdhm, dis_RuMP))

print(flux_range_selection(fva_bsh_wtg, ass_RuMP))
print(flux_range_selection(fva_bsh_wtm, ass_RuMP))
print(flux_range_selection(fva_bsh_mdhg, ass_RuMP))
print(flux_range_selection(fva_bsh_mdhm, ass_RuMP))


# vi prøver det samme med de andre fva-løsningene: 

print(flux_range_selection(fva_wtg, bsh_dependent_oxidation))
print(flux_range_selection(fva_wtm, bsh_dependent_oxidation))
print(flux_range_selection(fva_mdhg, bsh_dependent_oxidation))
print(flux_range_selection(fva_mdhm, bsh_dependent_oxidation))

print(flux_range_selection(fva_wtg, dis_RuMP))
print(flux_range_selection(fva_wtm, dis_RuMP))
print(flux_range_selection(fva_mdhg, dis_RuMP))
print(flux_range_selection(fva_mdhm, dis_RuMP))

# apparently der er positiv flux gjennom assimilatory rump for alle fva resultatene, men ikke når vi legger til bsh...
print(flux_range_selection(fva_wtg, ass_RuMP))
print(flux_range_selection(fva_wtm, ass_RuMP))
print(flux_range_selection(fva_mdhg, ass_RuMP))
print(flux_range_selection(fva_mdhm, ass_RuMP))

# så det er ingen positiv flux noe sted eller?

# We try to create a dataframe from our results that can be used to visualize only the 
# flux ranges that do not span 0: 

wtg_data = {"minimum": -2.3291057589174686, "maximum": -0.06835604604472031}

wtg_df = pd.DataFrame(wtg_data, index = ["R01056"])

print(wtg_df)

# Now we do the same for WTM

wtm_data = {"minimum": [-3.2952572078607076, 0.14419005411632568, 1.3070508571681785,
                       1.3070508571648183], "maximum": [-0.07628316571733507, 1.7708918339344546, 4.094469708395903, 4.094469708395903]}

wtm_df = pd.DataFrame(wtg_data, index = ['R01529', 'R01830', 'R05338', 'R05339'])

print(wtm_df)

# now we do the same for MDHG: 

mdhg_data = {"minimum": [-0.4924137674530814], "maximum": [-0.061764144950954517]}
             
mdhg_df = pd.DataFrame(wtg_data, index = ['R01056'])

print(mdhg_df)

# now we do the same for MDHM: 

mdhm_data = {"minimum": [0.1468594460237765,-1.725099793129659, 0.20881711639744305,
                        -4.438133595993175, 0.2434745116165616,0.602791143524074,0.602791143524074], 
             "maximum": [0.7105324676852716, -0.4522916280022473, 0.8452211989539047, 
                         -0.20881711640103107, 0.9029015498678215, 2.28289, 2.28289]}
             
mdhm_df = pd.DataFrame(wtg_data, index = ['R01056', 'R01529', 'R01641', 'R01827', 'R01830', 'R05338', 'R05339'])

print(mdhm_df)

# writing the min-fluxes to use to illustrate: 

write_min_flux_json(filename="wtg_range.json", fva_solution=wtg_df)


write_min_flux_json(filename="wtm_range.json", fva_solution=wtm_df)


write_min_flux_json(filename="mdhg_range.json", fva_solution=mdhg_df)


write_min_flux_json(filename="mdhm_range.json", fva_solution=mdhm_df)



                minimum    maximum
E00001         0.261640  22.234195
E00002        -7.029870  -4.323848
E00003       -14.568737  -0.215070
E00004         0.000000   5.794710
E00005         0.000000   7.176834
...                 ...        ...
rxn10198_new   0.000000   0.000000
bio00006_new   0.218500   0.230000
rxn42075       0.000000   0.000000
rxn44708       0.000000   0.000000
rxn46864       0.000000   0.000000

[1746 rows x 2 columns]
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'R01056': [-2.3291057589174686, -0.06835604604472031]}
{'R01529': [-3.2952572078607076, -0.07628316571733507], 'R01830': [0.14419005411632568, 1.7708918339344546], 'R05338': [1.3070508571681785, 4.094469708395903], 'R05339': [1.3070508571648183, 4.094469708395903]}
{'R01056': [-0.4924137674530814, -0.061764144950954517]}
{'R01056': [0.1468594460237765, 0.7105324676852716], 'R01529': [-1.725099793129659, -0.4522916280022473], 'R01641': [0.20881711639744305, 0.8452211989539047], 'R01827': [-

In [29]:
print(bsh_reactions)

for id in bsh_reactions:
    print(wtg.reactions.get_by_id(id).check_mass_balance())

['rxn42075', 'rxn44708', 'rxn46864', 'R00519']
{}
{}
{}
{}


In [30]:
print(fva_wtg)

def fva_positive_range(df):
    """
    fva_positive_range removes the rows where the flux range spans 0. 
    """
    for index, row in df.iterrows():
        if (row["minimum"] < 0 and row["maximum"] < 0) or (row["minimum"] > 0 and row["maximum"] > 0):
            print(index, row["minimum"], row["maximum"])
        

fva_wtg_removed = fva_positive_range(fva_wtg)

print(fva_wtg_removed)

                minimum    maximum
E00001         2.189119  21.385102
E00002        -7.029870  -4.378455
E00003       -14.475908  -0.215070
E00004         0.000000   5.794710
E00005         0.000000   7.130419
...                 ...        ...
rxn10201_new   0.000000   0.000000
rxn10200_new   0.000000   0.000000
rxn05296_new   0.126285   0.132931
rxn10198_new   0.000000   0.000000
bio00006_new   0.218500   0.230000

[1743 rows x 2 columns]
E00001 2.189119035072567 21.38510232360246
E00002 -7.02987 -4.3784552168068425
E00003 -14.475907603129011 -0.2150700303100924
E00006 -11.361394980497566 -0.09710928157364083
E00023 -2.6472975629262026 -0.03542690516129303
E00030 -0.0006860900000000003 -0.0006517854999999982
E00033 -29.900028055924093 -6.6565097169911995
E00084 -0.1512479999999998 -0.14368559999999983
E00096 -4.60114 -2.11533717015282
E00103 -0.021790200000000023 -0.020700690000000025
E00217 -0.0007380699999999996 -0.0007011664999999997
R00009 0.016658335176718886 2.619169825315393
R

In [31]:
# wtg

wtg_data = {"minimum": [-1,-1,1,-1,1], 
             "maximum": [-1,-1,1,-1,1]}
             
wtg_df = pd.DataFrame(wtg_data, index = ['R01056', 'R01529', 'R01641', 'R01827', "R01830"])

print(wtg_df)

write_min_flux_json("wtg_default.json", wtg_df)

# wtm 

wtm_data = {"minimum": [-1,1,1,1], 
             "maximum": [-1,1,1,1]}
             
wtm_df = pd.DataFrame(wtm_data, index = ['R01529', 'R01830', 'R05338', 'R05339'])

print(wtm_df)

write_min_flux_json("wtm_default.json", wtm_df)

# mdhg

mdhg_data = {"minimum": [-1], 
             "maximum": [-1]}

mdhg_df = pd.DataFrame(mdhg_data, index = ["R01056"])

print(mdhg_df)

write_min_flux_json("mdhg_default.json", mdhg_df)

# mdhm 

mdhm_data = {"minimum": [1,-1,1,-1,1,1,1], 
            "maximum": [1,-1,1,-1,1,1,1]}

mdhm_df = pd.DataFrame(mdhm_data, index = ["R01056", "R01529", "R01641", "R01827", "R01830", "R05338", "R05339"])

print(mdhm_df)

write_min_flux_json("mdhm_default.json", mdhm_df)


        minimum  maximum
R01056       -1       -1
R01529       -1       -1
R01641        1        1
R01827       -1       -1
R01830        1        1
        minimum  maximum
R01529       -1       -1
R01830        1        1
R05338        1        1
R05339        1        1
        minimum  maximum
R01056       -1       -1
        minimum  maximum
R01056        1        1
R01529       -1       -1
R01641        1        1
R01827       -1       -1
R01830        1        1
R05338        1        1
R05339        1        1
